In [1]:
import pandas as pd
import numpy as np
import time
from time import gmtime, strftime

In [2]:
%%time
s3_athena_file = 's3://lb.athena.queries.result/Unsaved/2021/06/13/41880f40-92d0-4dc8-a108-2fc58c9bff5d.csv'
data = pd.read_csv(s3_athena_file, error_bad_lines=False, dtype='str')

CPU times: user 19.7 s, sys: 2.89 s, total: 22.6 s
Wall time: 43.5 s


In [3]:
data.dropna(inplace=True)

print(data.shape)
print(data.columns)

data.head()

(1808399, 16)
Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'star_rating', 'helpful_votes',
       'total_votes', 'vine', 'verified_purchase', 'review_headline',
       'review_body', 'review_date', 'year', 'product_category'],
      dtype='object')


,marketplace,customer_id,review_id,product_id,product_parent,product_title,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,year,product_category
0,US,18684548,R3QK7ITZLJTWLF,B002BU0DPE,327243299,PS3 Ear Force PX21 Gaming Headset,1,0,0,N,Y,Cheaply made.,The headset is shoddy. The wires that transmit...,15520,2012,Video_Games
1,US,33788217,R2FNT507ZCB4GE,B001GM6C26,577190767,Mortimer Beckett and the Secrets of Spooky Man...,3,1,1,N,N,not bad,being a fan of hidden pictures this game can b...,14382,2009,Video_Games
2,US,17015645,R1LVPFY1GV1NLA,B00178630A,111226018,Diablo III - PC/Mac,1,6,8,N,N,"Addictive, but not fun",Let me give you a quick summary of the game. ...,15521,2012,Video_Games
3,US,32737971,R3MGDCF0BFTYQM,B0012NAXE2,215020735,Madden NFL 09,2,2,4,N,N,THE POOREST MADDEN GAME EVER,"When I first got this game for Christmas, I th...",14382,2009,Video_Games
4,US,41771473,RZD8INNNJ46OZ,B0038TR60M,598858515,Crimson Red & Black - Nintendo DS Lite Complet...,2,0,0,N,Y,Good case but still disappointed,The case is excellent. I'm well on my way to f...,15521,2012,Video_Games


In [4]:
%%time
data['review_body'] = data['review_headline'] + ' ' + data['review_body']

CPU times: user 993 ms, sys: 470 ms, total: 1.46 s
Wall time: 1.58 s


In [5]:
%%time
data = data[['review_id', 'product_id','star_rating','review_body']]

CPU times: user 79.2 ms, sys: 21.9 ms, total: 101 ms
Wall time: 122 ms


In [7]:
data.head()

,review_id,product_id,star_rating,review_body
0,R3QK7ITZLJTWLF,B002BU0DPE,1,Cheaply made. The headset is shoddy. The wires...
1,R2FNT507ZCB4GE,B001GM6C26,3,not bad being a fan of hidden pictures this ga...
2,R1LVPFY1GV1NLA,B00178630A,1,"Addictive, but not fun Let me give you a quick..."
3,R3MGDCF0BFTYQM,B0012NAXE2,2,THE POOREST MADDEN GAME EVER When I first got ...
4,RZD8INNNJ46OZ,B0038TR60M,2,Good case but still disappointed The case is e...


In [8]:
data.star_rating.unique()

array(['1', '3', '2', '4', '5'], dtype=object)

In [9]:
%%time
data['label'] = data.star_rating.map({
    '1':'__label__negative__',
    '2':'__label__negative__',
    '3':'__label__neutral__',
    '4':'__label__positive__',
    '5':'__label__positive__'
})

CPU times: user 174 ms, sys: 3.64 ms, total: 177 ms
Wall time: 224 ms


In [10]:
data.head()

,review_id,product_id,star_rating,review_body,label
0,R3QK7ITZLJTWLF,B002BU0DPE,1,Cheaply made. The headset is shoddy. The wires...,__label__negative__
1,R2FNT507ZCB4GE,B001GM6C26,3,not bad being a fan of hidden pictures this ga...,__label__neutral__
2,R1LVPFY1GV1NLA,B00178630A,1,"Addictive, but not fun Let me give you a quick...",__label__negative__
3,R3MGDCF0BFTYQM,B0012NAXE2,2,THE POOREST MADDEN GAME EVER When I first got ...,__label__negative__
4,RZD8INNNJ46OZ,B0038TR60M,2,Good case but still disappointed The case is e...,__label__negative__


## Tokenize reviews

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
%%time
data['review_body'] = data['review_body'].apply(nltk.word_tokenize)

In [ ]:
%%time
data['review_body'] = data.apply(lambda row: " ".join(row['review_body']).lower(), axis=1)